In [1]:
import os

In [2]:
%pwd


'c:\\Users\\devan\\Downloads\\Chicken classification\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'c:\\Users\\devan\\Downloads\\Chicken classification'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [14]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [31]:
import time


In [32]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [39]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [52]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        # 1) load without using the old compile() settings
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )
        # 2) now compile fresh, forcing eager execution
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),             # ← or your original optimizer
            loss="categorical_crossentropy",                  # ← your original loss
            metrics=["accuracy"],                             # ← your original metrics
            run_eagerly=True                                  # ← THIS makes .fit() 100% eager
        )
    
    def train_valid_generator(self):
        # … exactly as before …
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            class_mode="categorical",   # ← be explicit about class_mode
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            class_mode="categorical",   # ← explicit here as well
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        # ensure generators are ready
        self.steps_per_epoch    = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps   = self.valid_generator.samples // self.valid_generator.batch_size

        # debug print to confirm eager
        import tensorflow as tf
        print(">>> tf.executing_eagerly():", tf.executing_eagerly())
        print(">>> model.run_eagerly flag:", self.model.run_eagerly)

        # now call fit — it will run entirely in eager mode
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [54]:
try:
    config = ConfigurationManager()
    
    # Prepare callbacks
    prepare_callbacks = PrepareCallback(config=config.get_prepare_callback_config())
    callback_list    = prepare_callbacks.get_tb_ckpt_callbacks()

    # Training
    training = Training(config=config.get_training_config())
    training.get_base_model()           # loads & recompiles with run_eagerly=True
    training.train_valid_generator()    # sets up your ImageDataGenerators
    
    training.train(callback_list=callback_list)  # kicks off .fit()

except Exception as e:
    raise e


[2025-08-07 23:12:19,766: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-07 23:12:19,771: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-07 23:12:19,773: INFO: common: created directory at: artifacts]
[2025-08-07 23:12:19,774: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-08-07 23:12:19,776: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-08-07 23:12:19,778: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
>>> tf.executing_eagerly(): True
>>> model.run_eagerly flag: True
Epoch 1/2
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6089 - loss: 1.0802[2025-08-07 23:13:36,883: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Ker

c:\Users\devan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 296ms/step - accuracy: 0.9375 - loss: 0.2844 - val_accuracy: 0.9062 - val_loss: 0.3348
[2025-08-07 23:13:46,208: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
